# NiO VASP
Simple example demonstrating how to use `pytheos` to set up and analyze rocksalt NiO with various different magnetic orderings.

## Relaxation Calculation Set Up
First we need to set up three calculations:
1. NiO unit cell (nonmagnetic)
2. NiO unit cell (ferromagnetic)
3. NiO 2x2x2 supercell (antiferromagnetic within 111 planes)

NOTE that the r2scan meta-gga and conventional 8-atom rocksalt unit cells are used in all cases here...

In [1]:
from ase.build import bulk
from pytheos.vasp.inputs import CalcInputs
from pytheos.structure import generation
import os

unit_cell = bulk(name="NiO", crystalstructure="rocksalt", a = 4.20, cubic=True)

In [2]:
# set up NM relaxation
nm_calc = CalcInputs(structure=unit_cell)

nm_calc.update_incar({
    "ISPIN": 1, # turn off spin polarization
    "KPAR": 4 # to speed up calculation
    })

nm_calc.write_files(output_dir="NiO_vasp_calcs/relax_nm")
os.system("cp ../scripts/submitvasp NiO_vasp_calcs/relax_nm/submitvasp")
os.system("cp ../scripts/cstdn_vasp_doublerelax.py NiO_vasp_calcs/relax_nm/cstdn.py")

/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/io/vasp/sets.py:486: BadInputSetWarning: POTCAR data with symbol Ni_pv is not known by pymatgen to correspond with the selected user_potcar_functional='PBE_54'. This POTCAR is known to correspond with functionals ['PBE_54_W_HASH', 'PBE_64', 'PBE_52_W_HASH']. Please verify that you are using the right POTCARs!
  potcar="\n".join(self.potcar_symbols) if potcar_spec else self.potcar,
/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/io/vasp/sets.py:486: BadInputSetWarning: POTCAR data with symbol O is not known by pymatgen to correspond with the selected user_potcar_functional='PBE_54'. This POTCAR is known to correspond with functionals ['PBE_54_W_HASH', 'PBE_64', 'PBE_52_W_HASH']. Please verify that you are using the right POTCARs!
  potcar="\n".join(self.potcar_symbols) if potcar_spec else self.potcar,


0

In [3]:
# set up FM relaxation
fm_calc = CalcInputs(structure=unit_cell)

fm_calc.update_incar({
    "KPAR": 4 # to speed up calculation
    })

fm_calc.write_files(output_dir="NiO_vasp_calcs/relax_fm")
os.system("cp ../scripts/submitvasp NiO_vasp_calcs/relax_fm/submitvasp")
os.system("cp ../scripts/cstdn_vasp_doublerelax.py NiO_vasp_calcs/relax_fm/cstdn.py")

0

In [4]:
# set up AFM relaxation
# to accomodate for the AFMII magnetic ordering, we need to make a 2x2x2 supercell
super_cell = generation.make_supercell(structure=unit_cell, dimensions=(2, 2, 2))

afm_calc = CalcInputs(structure=super_cell)

afm_calc.update_incar({
    "KPAR": 2 # to speed up calculation
    })

afm_calc.apply_mag_order(
    magmom_values={"Ni": 3, "O": 0}, 
    mag_order_file="magorder_NiO_AFMII.yaml", # read in magorder*.yaml file, which can accomodate any magnetic ordering you want to test
    rattle_amount=0.0, # not rattling the initial magnetic moments for now
    )

afm_calc.write_files(output_dir="NiO_vasp_calcs/relax_afm")
os.system("cp ../scripts/submitvasp NiO_vasp_calcs/relax_afm/submitvasp")
os.system("cp ../scripts/cstdn_vasp_doublerelax.py NiO_vasp_calcs/relax_afm/cstdn.py")

0

Take a look at the differences in the INCAR files for each calculation (`ISPIN` and `MAGMOM` specifically)

## Density of States Calculation Setup
Once you have run these relaxations, we will using the `pytheos.vasp.modifiers` module to modify these relaxation calculations into density of states (DOS) calculations. It is best practice to only compare energies taken from static, DOS calculations (with `ISMEAR = -5` in the INCAR) so we will go ahead and do this for our magnetic order comparison here.
- NOTE that I have downloaded the following output files for each calculation that won't be stored in GitHub: vasprun.xml, EIGENVAL, CONTCAR, CHGCAR, WAVECAR

In [5]:
from pytheos.vasp.modifiers import CalcModifier